## Semi-mechanistic modelling exploration

### Rationale
Thinking about this equation in Faria, et al:
$\\i_{s,t} = (1-\frac{n_{s,t}}{N})R_{s,t}\sum_{\tau<t} i_{s,\tau}g_{t-\tau}$

For me, this is a more standard "semi-mechanistic" modelling approach,
in that the population is not explicitly partitioned into categories or compartments.
It is partitioned in this way for our standard compartmental models,
including both standard SEIR `summer` models, 
as well as Romain's semi-mechanistic models,
which are compartmental with an additional non-mechanistic random walk 
flow adjustment.

First, ignoring strains, I'll consider:
$\\i_t = (1-\frac{n_t}{N})R_t\sum_{\tau<t} i_{\tau}g_{t-\tau}$

For now, I'll also ignore susceptible depletion and a varying reproduction number, and so consider:
$\\i_t = R_0\sum_{\tau<t} i_\tau g_{t-\tau}$

This notebook builds up the basic code from the first principles,
checking with each extension that the results we are getting back are the same
as in the previous, more explicit version.

In [ ]:
from typing import Dict
from scipy.stats import gamma
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'

### Parameters
Choose some arbitrary model parameters to get started.

In [ ]:
n_times = 20
seed = 1.0
r0 = 2.0
incidence = np.zeros(n_times)
incidence[0] = seed

### Generation time
Get a distribution we can sensibly use for the generation time,
which could represent an acute immunising respiratory infection.

In [ ]:
def get_gamma_params_from_mean_sd(req_mean: float, req_sd: float) -> Dict[str, float]:
    var = req_sd ** 2.0
    scale = var / req_mean
    a = req_mean / scale
    return {'a': a, 'scale': scale}

# Generation time parameters
req_sd = 1.5
req_mean = 5.0
gamma_params = get_gamma_params_from_mean_sd(req_mean, req_sd)

# Get the increment in the CDF
# (i.e. the integral over the increment by one in the distribution)
gen_time_densities = np.diff(gamma.cdf(range(n_times + 1), **gamma_params))

pd.Series(gen_time_densities, index=range(n_times)).plot()

### Check calculations make sense from first principles
Looping in Python to be completely explicit (with pre-calculated generation times).

In [ ]:
for t in range(1, n_times):
    val = 0
    for tau in range(t):  # For each day preceding the day of interest
        delay = t - tau - 1  # The generation time index for each preceding day to the day of interest
        val += incidence[tau] * gen_time_densities[delay] * r0  # Calculate the incidence value
    incidence[t] = val
incidence

Get rid of one loop to get lists/arrays for the incidence and generation time distribution 
(and check that calculations are the same).

In [ ]:
for t in range(1, n_times):
    delays = [t - tau - 1 for tau in range(t)]
    gammas = gen_time_densities[delays]
    incidence[t] = (incidence[:t] * gammas).sum() * r0
incidence

We can get this down to a one-liner if preferred. This is going to just keep going up exponentially, of course, because $R_{0} > 1$ and there is no susceptible depletion.

In [ ]:
for t in range(1, n_times):
    incidence[t] = (incidence[:t] * gen_time_densities[:t][::-1]).sum() * r0
incidence
pd.Series(incidence).plot(labels={'index': 'day', 'value': 'incidence'})

Already some interesting phenomena there, 
in that the humps are the generations of cases from the first seeding infection,
which progressively smooth into one-another with generations of cases.

### Threshold behaviour
Next let's check that the threshold behaviour is approximately correct.
We would expect a declining epidemic with $R_{0} < 1$ even without
susceptible depletion.

In [ ]:
r0 = 0.8
for t in range(1, n_times):
    incidence[t] = (incidence[:t] * gen_time_densities[:t][::-1]).sum() * r0
incidence
pd.Series(incidence).plot(labels={'index': 'day', 'value': 'incidence'})